# O Problema

Neste estudo de caso o nosso ambiente é uma lanchonete localizada na interseção entre 3 grandes supermercados da cidade. Por estar em um bom local, o fluxo de pessoas pela loja é bem intenso, composto principalmente de pessoas que estão realizando suas compras e param para um lanche rápido.

Dado este contexto, a lanchonete quer entender qual a melhor forma de montar o cardápio de salgados para maximizar o lucro líquido do dia. Ela conta com dois tipos principais de salgados, os fritos e os assados. O portfólio de fritos é composto por coxinha e pasteis de carne e de frango. Já os assados são joelho, empadinha, pão de queijo e esfirras de carne e de frango.

Cada salgado possui uma receita específica, detalhadas no documento JSON chamado *receitas.json*. No grosso modo estas receitas compartilham trigo, leite, óleo e fermento como matérias-prima. O fluxo de compra diário deste materiais pode ser feito por demanda ou de forma fixada, hoje existem estas duas possibilidades.

Vamos explorar nos próximos tópicos algumas análises e abordagens de otimização, sigamos em frente.

## Restrições

Pelo histórico da lanchonete, foi encontrado que é possível se vender em média 300 salgados fritos e 400 salgados assados por dia, então:

$ \sum_{i=1}^n x_i \leq 300 $

$ \sum_{j=1}^m x_j \leq 400 $

Em que i pertence ao conjunto de salgados fritos e j ao conjunto de salgados assados.

Esta é a restrição base do nosso estudo de caso. Porém, em algumas aplicações trabalharemos com as restrições de matérias-prima, confrontando com a não utilização das mesmas para a otimização. Neste momento, a ideia é avaliar se é possível reduzir o desperdício e o custo dos ingredientes.

In [1]:
import os
import json
import random
from typing import List

BASE_PATH = os.path.dirname(os.getcwd())
RECIPES_PATH = f"{BASE_PATH}/data/receitas.json"

In [2]:
with open(RECIPES_PATH) as file:
    recipes = json.load(file)

In [3]:
def make_possibilities(recipes: dict):
    fried = []
    roast = []
    for product in recipes["salgados"]:
        if product["tipo"] == "Frito":
            fried.append(product)
        elif product["tipo"] == "Assado":
            roast.append(product)

    return fried, roast

In [4]:
def random_possibilty(fried: List[dict], roast: List[dict], max_fried: int, max_roast: int):
    solution_fried = []
    solution_roast = []

    for _ in range(0, len(fried)):
        value = random.randint(0, int(max_fried * 0.5))
        if sum(solution_fried) + value > max_fried:
            solution_fried.append(max_fried - sum(solution_fried))
        else:
            solution_fried.append(value)
    
    for _ in range(0, len(roast)):
        value = random.randint(0, int(max_roast * 0.5))
        if sum(solution_roast) + value > max_roast:
            solution_roast.append(max_roast - sum(solution_roast))
        else:
            solution_roast.append(value)

    return solution_fried, solution_roast

In [25]:
def calc_profit(recipes, fried, roast, solution_fried, solution_roast) -> float:
    cost, revenue = [], []
    stock = dict()
    for material in recipes["materias_prima"]:
        stock[material["nome"]] = 0

    def find_material(name: str):
        result = None
        for material in recipes["materias_prima"]:
            if material["nome"] == name:
                result = material
        return result
    
    def execute_buy(qntd: float, product_name: str) -> None:
        product = find_material(product_name)
        if product["compra_fracionada"]:
            stock[product_name] = stock[product_name] + qntd
            value = (qntd * product["custo_unitario"] / product["quantidade_unitaria"])
        else:
            stock[product_name] = stock[product_name] + product["quantidade_unitaria"]
            value = product["custo_unitario"]
        
        cost.append(value)

    def consume_material(qntd: float, product_name: str) -> None:
        stock[product_name] = stock[product_name] - qntd
    
    for i in range(0, len(fried)):
        # Calcula a receita de venda
        units = solution_fried[i]
        revenue.append(units * fried[i]["vendido_por"])

        # Computa os custos para os salgados fritos
        for material in fried[i]["ingredientes"]:
            # Encontra a quantidade necessária do material
            qntd = material["quantidade"] * units / fried[i]["rendimento"]

            # Faz uma compra até ter a quantidade necessária para a receita
            while stock[material["nome"]] < qntd:
                execute_buy(qntd, material["nome"])

            # Consome o material
            consume_material(qntd, material["nome"])

    for i in range(0, len(roast)):
        # Calcula a receita de venda
        units = solution_roast[i]
        revenue.append(units * roast[i]["vendido_por"])

        # Computa os custos para os salgados assados
        for material in roast[i]["ingredientes"]:
            # Encontra a quantidade necessária do material
            qntd = material["quantidade"] * units / roast[i]["rendimento"]

            # Faz uma compra até ter a quantidade necessária para a receita
            while stock[material["nome"]] < qntd:
                execute_buy(qntd, material["nome"])

            # Consome o material
            consume_material(qntd, material["nome"])
                
    return sum(revenue) - sum(cost)

In [146]:
def get_near_solutions(solution: List[int], sum_limit: int, number=50) -> List[int]:
    solutions: List[int] = []
    for _ in range(0, number):
        while True:
            i = random.randint(0, len(solution) - 1)
            step = int(0.3 * sum_limit)
            new_solution = solution.copy()
            new_solution[i] = new_solution[i] + random.randint(-1 * step, step)
            if sum(new_solution) <= sum_limit:
                break
        solutions.append(new_solution)
    return solutions

In [168]:
fried, roast = make_possibilities(recipes)
max_fried, max_roast = 300, 400

solution_fried, solution_roast = random_possibilty(fried, roast, max_fried, max_roast)
profit = calc_profit(recipes, fried, roast, solution_fried, solution_roast)

i = 0
neighbors = 500

current_fried_solution = solution_fried
current_roast_solution = solution_roast
current_profit = profit

while True:
    print(f"Iteração {i} do algoritmo Subida de Encosta", end="\r")
    improved = False

    fried_near_solutions = get_near_solutions(current_fried_solution, max_fried, number=neighbors)
    roast_near_solutions = get_near_solutions(current_roast_solution, max_roast, number=neighbors)

    for new_fried_solution, new_roast_solution in zip(fried_near_solutions, roast_near_solutions):
        new_profit = calc_profit(recipes, fried, roast, new_fried_solution, new_roast_solution)
        if new_profit > current_profit:
            current_fried_solution = new_fried_solution
            current_roast_solution = new_roast_solution
            current_profit = new_profit
            improved = True

    if not improved:
        break

    i = i + 1

print(f"\nMaior lucro encontrado: R$ {round(current_profit, 2)}")
print(current_fried_solution, current_roast_solution)

Iteração 3 do algoritmo Subida de Encosta
Maior lucro encontrado: R$ 3704.22
[110, 134, 52] [150, 179, 18, 45, 8]


In [138]:
solution_fried, fried_near_solutions

([138, 65, 56],
 [[138, 65, 96],
  [138, 101, 56],
  [138, 65, 84],
  [138, 102, 56],
  [138, 85, 56],
  [138, 65, 85],
  [104, 65, 56],
  [138, 65, 27],
  [138, 65, 63],
  [123, 65, 56],
  [138, 65, 46],
  [138, 42, 56],
  [138, 65, 70],
  [166, 65, 56],
  [138, 96, 56],
  [138, 65, 51],
  [138, 33, 56],
  [80, 65, 56],
  [138, 65, 56],
  [108, 65, 56],
  [109, 65, 56],
  [138, 65, 39],
  [138, 65, 55],
  [150, 65, 56],
  [138, 65, 64],
  [156, 65, 56],
  [104, 65, 56],
  [138, 101, 56],
  [138, 40, 56],
  [138, 21, 56],
  [129, 65, 56],
  [86, 65, 56],
  [138, 64, 56],
  [138, 95, 56],
  [138, 103, 56],
  [138, 90, 56],
  [138, 36, 56],
  [138, 65, 46],
  [138, 65, 9],
  [163, 65, 56],
  [138, 65, 84],
  [157, 65, 56],
  [91, 65, 56],
  [138, 65, 25],
  [138, 31, 56],
  [138, 65, 95],
  [132, 65, 56],
  [138, 65, 70],
  [138, 65, 84],
  [138, 65, 49],
  [175, 65, 56],
  [138, 15, 56],
  [138, 65, 2],
  [138, 58, 56],
  [79, 65, 56],
  [80, 65, 56],
  [105, 65, 56],
  [138, 65, 64],
 

In [60]:
print(f"Maior lucro encontrado: R$ {round(current_profit, 2)}")
print(current_fried_solution, current_roast_solution)

Maior lucro encontrado: R$ 3264.6
[105, 65, 51] [106, 98, 46, 60, 77]


In [21]:
for product in recipes["salgados"]:
    total_cost = 0
    for material in product["ingredientes"]:
        value = 0
        for item in recipes["materias_prima"]:
            if item["nome"] == material["nome"]:
                value = (material["quantidade"] / item["quantidade_unitaria"]) * item["custo_unitario"]
        total_cost = total_cost + value

    print("{}: {}".format(product["nome"], product["tipo"]))
    print(f"Custo total: R$ {round(total_cost, 2)}")
    print(f"Custo unitário: R$ {round(total_cost / product['rendimento'], 2)}")
    print(f"Vendido por R$ {round(product['vendido_por'], 2)}")
    print()

Coxinha: Frito
Custo total: R$ 10.18
Custo unitário: R$ 0.68
Vendido por R$ 6

Joelho: Assado
Custo total: R$ 25.21
Custo unitário: R$ 1.68
Vendido por R$ 7

Empadinha: Assado
Custo total: R$ 15.31
Custo unitário: R$ 1.53
Vendido por R$ 8

Pastel de carne: Frito
Custo total: R$ 22.84
Custo unitário: R$ 1.52
Vendido por R$ 6

Pastel de frango: Frito
Custo total: R$ 15.39
Custo unitário: R$ 1.03
Vendido por R$ 6

Esfirra de carne: Assado
Custo total: R$ 18.77
Custo unitário: R$ 1.88
Vendido por R$ 7

Esfirra de frango: Assado
Custo total: R$ 11.31
Custo unitário: R$ 1.13
Vendido por R$ 7

Pão de queijo: Assado
Custo total: R$ 39.74
Custo unitário: R$ 1.32
Vendido por R$ 7

